# Population Data from CSV

This notebooks reads sample population data from `data/atlantis.csv` and plots it using Matplotlib. Edit `data/atlantis.csv` and re-run this cell to see how the plots change!

In [1]:
pip install "ray[rllib]" tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install swig

In [ ]:
pip install gymnasium[box2d]

In [ ]:
!pip install wandb

In [2]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import random
import wandb

#мы заменили float и np.array
class GoLeftEnv(gym.Env):
  """
  Custom Environment that follows gym interface.
  This is a simple env where the agent must learn to go always left.
  """
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {'render.modes': ['console']}
  # Define constants for clearer code
  FIRST = 0
  SECOND = 1
  THIRD = 2
  FORTH = 3
  FIFTH= 4
  SIXTH = 5
  SEVENTH = 6

  def __init__(self, grid_size=10, render_mode="console"):
    super(GoLeftEnv, self).__init__()
    self.render_mode = render_mode

    self.fields = np.array([[0, 0, 7, 7,], [1, 0, 0, 0,], [2, 0, 0, 0,], [3, 0, 0, 0,], [4, 1, 0, 0,], [5, 0, 0, 0], [6, 0, 0, 0], [7, 0, 0, 0], [8, 1, 0, 0], [9, 0, 0, 0], [10, 0, 0, 0], [11, 0, 0, 0], [12, 0, 0, 0],
                   [13, 0, 0, 0], [14, 1, 0, 0], [15, 0, 0, 0], [0, 0, 0, 0]]).flatten('C')

    # Define action and observation space
    # They must be gym.spaces objects
    # Example when using discrete actions, we have two: left and right
    n_actions = 7
    self.action_space = spaces.Discrete(n_actions)
    # The observation will be the coordinate of the agent
    # this can be described both by Discrete and Box space
    self.observation_space = spaces.Box(low=-100, high=100,
                                        shape=(68,), dtype=np.float32)
  #попробовать переделать всё в np.array

  def reset(self, seed=None, options=None):
    """
    Important: the observation must be a numpy array
    :return: (np.array)
    """
    super().reset(seed=seed, options=options)


    # Initialize the agent at the right of the grid
    self.fields = np.array([[0, 0, 7, 7,], [1, 0, 0, 0,], [2, 0, 0, 0,], [3, 0, 0, 0,], [4, 1, 0, 0,], [5, 0, 0, 0], [6, 0, 0, 0], [7, 0, 0, 0], [8, 1, 0, 0], [9, 0, 0, 0], [10, 0, 0, 0], [11, 0, 0, 0], [12, 0, 0, 0],
                   [13, 0, 0, 0], [14, 1, 0, 0], [15, 0, 0, 0], [0, 0, 0, 0]])



    # here we convert to float32 to make it more general (in case we want to use continuous actions)
    return self.fields.astype(np.float32).flatten('C'), {}

  def step(self, action):

     self.fields.reshape(17, 4)

     def first():
            numberforcycles = 15

            balcycle1 = 0

            roll = self.fields[16][0]

            lst = [0, 1, 2, 3, 4]
            weights = [6.25, 25, 37.5, 25, 6.25]
            nextroll = random.choices(lst, weights=weights, k=1)
            nextroll = nextroll[0]
            self.fields[16][0] = nextroll
            #print("Выпала" , roll)
            arrplayer1 = []
            while balcycle1 < numberforcycles:
              ballnow = self.fields[balcycle1][2]
              ballindex = self.fields[balcycle1][0]
          #расположение фишек игрока 1
              if ballnow != 0:
               arrplayer1.append(ballindex)
              balcycle1 += 1
            if self.fields[8][3] == 1:
                arrplayer1.append(8)
            #print(arrplayer1)

            balcycle1 = 0
            arrchoice1 = []
            while balcycle1 < numberforcycles:
              ballnow = self.fields[balcycle1][2]
              ballindex = self.fields[balcycle1][0]
              if ballnow != 0:
                ballfuture = ballindex + roll
                if ballfuture <= numberforcycles:
                 if ballfuture not in arrplayer1:
                  arrchoice1.append(ballfuture)
              # print("Выпала" , self.roll)
                  #print("текущая позиция" , ballindex)
                  #print("возможная позиция" , ballfuture)
              balcycle1 += 1



            if arrchoice1:

              #случайный бот:
              playeronemove = random.choice(arrchoice1)

              #Очень жадный бот:
              #playeronemove = len(arrchoice1)
              #playeronemove = arrchoice1[playeronemove-1]

              #Жадный бот:
              #playeronemove = 33
              #for arrchoice in arrchoice1:
              #  if self.fields[arrchoice][1] == 1 or (self.fields[arrchoice][3] == 1 and self.fields[arrchoice][0] >= 5 and self.fields[arrchoice][0] <= 12):
              #    playeronemove = self.fields[arrchoice][0]
              #if playeronemove == 33:
              #  playeronemove = len(arrchoice1)
              #  playeronemove = arrchoice1[playeronemove-1]

              #playeronemove = random.choice(arrchoice1)
              playeronepos = playeronemove - roll
              #print(playeronepos)
              self.fields[playeronepos][2] = self.fields[playeronepos][2] - 1
              self.fields[playeronemove][2] = self.fields[playeronemove][2] + 1
              if self.fields[playeronemove][3] == 1 and playeronemove >= 5 and playeronemove <= 12:
                self.fields[playeronemove][3] = 0
                self.fields[0][3] = self.fields[0][3] + 1
                #print("мы забрали шашку на" , self.fields[playeronemove][0])
              #print("боту выпал", roll)
              #print("бот ходит", playeronemove)
              return playeronemove

     def second():
            numberforcycles = 15

            agentroll = self.fields[16][1]


            lst = [0, 1, 2, 3, 4]
            weights = [6.25, 25, 37.5, 25, 6.25]
            nextagentroll = random.choices(lst, weights=weights, k=1)
            nextagentroll = nextagentroll[0]
            self.fields[16][1] = nextagentroll
            #print("Агенту Выпал" , agentroll)

            agentcycle = 0
            arrplayeragent = []

            while agentcycle < numberforcycles:
              agentballnow = self.fields[agentcycle][3]
              agentballindex = self.fields[agentcycle][0]
          #расположение фишек игрока 2
              if agentballnow != 0:
               arrplayeragent.append(agentballindex)
              agentcycle += 1
            if self.fields[8][2] == 1:
                arrplayeragent.append(8)

            #print("фишки агента", arrplayeragent)

            agentcycle = 0
            arrplayeragent2 = []

            while agentcycle < numberforcycles:
              agentballnow = self.fields[agentcycle][3]
              agentballindex = self.fields[agentcycle][0]
          #будущее фишек игрока 2
              if agentballnow != 0:
               agentballfuture = agentballindex + agentroll
               if agentballfuture <= numberforcycles:
                if agentballfuture not in arrplayeragent:
                  arrplayeragent2.append(agentballfuture)
              agentcycle += 1

            #print(arrplayeragent2)



            if arrplayeragent2:
              playeragentmove = len(arrplayeragent2)
              playeragentmove = arrplayeragent2[0]#arrplayeragent2[playeragentmove-1]

              if action == self.FIRST:
                if len(arrplayeragent2) > 0:
                 playeragentmove = arrplayeragent2[0]
                else:
                 playeragentmove = playeragentmove
              elif action == self.SECOND:
               if len(arrplayeragent2) > 1:
                playeragentmove = arrplayeragent2[1]
               else:
                playeragentmove = playeragentmove
              elif action == self.THIRD:
               if len(arrplayeragent2) > 2:
                playeragentmove = arrplayeragent2[2]
               else:
                playeragentmove = playeragentmove
              elif action == self.FORTH:
               if len(arrplayeragent2) > 3:
                playeragentmove = arrplayeragent2[3]
               else:
                playeragentmove = playeragentmove
              elif action == self.FIFTH:
               if len(arrplayeragent2) > 4:
                playeragentmove = arrplayeragent2[4]
               else:
                playeragentmove = playeragentmove
              elif action == self.SIXTH:
               if len(arrplayeragent2) > 5:
                playeragentmove = arrplayeragent2[5]
               else:
                playeragentmove = playeragentmove
              elif action == self.SEVENTH:
               if len(arrplayeragent2) > 6:
                playeragentmove = arrplayeragent2[6]
               else:
                playeragentmove = playeragentmove
              else:
                raise ValueError("Received invalid action={} which is not part of the action space".format(action))



              #print("Агент делает выбор", playeragentmove)

              #playeragentmove = random.choice(arrplayeragent)
              playeragentpos = playeragentmove - agentroll
              self.fields[playeragentpos][3] = self.fields[playeragentpos][3] - 1
              self.fields[playeragentmove][3] = self.fields[playeragentmove][3] + 1
              if self.fields[playeragentmove][2] == 1 and playeragentmove >= 5 and playeragentmove <= 12:
                self.fields[playeragentmove][2] = 0
                self.fields[0][2] = self.fields[0][2] + 1
                #print("Агент забрал шашку на" , self.fields[playeragentmove][0])
              #print("Агенту выпал" , agentroll)
              #print("Агент ходит" , playeragentmove)
              return playeragentmove
            else:
              rrr = 0#print("Выпал 0 и агент не делает выбора")
     #while True:
     # thisbotmove = first()
     # if thisbotmove != 4 or thisbotmove != 8 or thisbotmove != 14:
     #    #print("нет второго хода бота")
     #    break

     if self.fields[16][3] != 1:
      thisbotmove = first()
      #print(first())
      if thisbotmove == 4 or thisbotmove == 8 or thisbotmove == 14:
       #print("Второй ход бота", thisbotmove)
       self.fields[16][2] = 1
      else:
       self.fields[16][2] = 0
    # if thisbotmove == 4 or thisbotmove == 8 or thisbotmove == 14:
     if self.fields[16][2] != 1:
      thisagentmove = second()
      #print(second())
      if thisagentmove == 4 or thisagentmove == 8 or thisagentmove == 14:
        #print("Второй ход агента", thisagentmove)
        self.fields[16][3] = 1
      else:
        self.fields[16][3] = 0
     #while True:
     # thisagentmove =  second()
     # if thisagentmove != 4 or thisagentmove != 8 or thisagentmove != 14:
     #    #print("нет второго хода агента")
     #    break


     terminated = bool(self.fields[15][2] >= 7 or self.fields[15][3] >= 7)
     truncated = False  # we do not limit the number of steps here

     reward = 0
     if self.fields[15][3] >= 7 and self.fields[15][2] != 7:
       reward = 1

    # Optionally we can pass additional info, we are not using that for now
     info = {}

     return np.array(self.fields).astype(np.float32).flatten('C'), reward, terminated, truncated, info


  def render(self):
        # agent is represented as a cross, rest as a dot
        if self.render_mode == "console":
            1==1

    #print(self.fields[0])

  def close(self):
    pass

In [3]:
import ray

ray.init()

2023-08-18 22:12:00,920	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-18 22:12:03,708	WARNING services.py:1832 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.61gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-08-18 22:12:04,180	INFO worker.py:1621 -- Started a local Ray instance.


Python version:,3.10.8
Ray version:,2.6.1


In [4]:
from ray import tune

# Register , how your env should be constructed (always with 5, or you can take values from the `config` EnvContext object):
tune.register_env("my_env", lambda config: GoLeftEnv())

2023-08-18 22:12:11,340	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
from ray import tune

tune.run("PPO",
         config={"env": "my_env",
                 "evaluation_interval": 2,
                 "evaluation_num_episodes": 20,
                 "num_workers": 1,
                 # other configurations go here, if none provided, then default configurations will be used
                 },
         local_dir="Urrnow",
         checkpoint_freq=5
         )

2023-08-18 22:12:30,926	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/codespace/.python/current/lib/python3.10/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
2023-08-18 22:12:30,927	WARNING syncer.py:260 -- You are using remote storage, but you don't have `fsspec` installed. This can lead to inefficient syncing behavior. To avoid this, install fsspec with `pip install fsspec`. Depending on your remote storage provider, consider installing the respective fsspec-package (see https://github.com/fsspec).
2023-08-18 22:12:52,699	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will

2023-08-18 22:12:53,213	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-18 22:12:53,214	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=7864) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=7864) 2023-08-18 22:12:57,805	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. Wh

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_my_env_60598_00000,420000,"{'ObsPreprocessorConnector_ms': 0.004522562026977539, 'StateBufferConnector_ms': 0.0033309459686279297, 'ViewRequirementAgentConnector_ms': 0.2528705596923828}","{'num_env_steps_sampled': 420000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 420000, 'num_agent_steps_trained': 0}",{},93.16,{},1,0.57,0,43,"{'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 0.03950502575305142}, 'default_policy': {'total_loss': 0.03950502575305142, 'policy_loss': -0.015486317728437594, 'vf_loss': 0.052967449719173665, 'vf_loss_unclipped': 0.052967449719173665, 'vf_explained_var': 0.14702792742104928, 'entropy': 0.33840868542634095, 'mean_kl_loss': 0.010119468702149399, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.20000000298023224}}, 'num_env_steps_sampled': 420000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 420000, 'num_agent_steps_trained': 0}",420000,0,420000,4000,251.136,0,0,0,0,1,0,0,0,"{'cpu_util_percent': 30.504347826086956, 'ram_util_percent': 24.36086956521739}",{},{},{},"{'mean_raw_obs_processing_ms': 0.4404442296553175, 'mean_inference_ms': 1.2645802029971827, 'mean_action_processing_ms': 0.1045043056800169, 'mean_env_wait_ms': 0.0883236720705588, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 1.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 0.57, 'episode_len_mean': 93.16, 'episode_media': {}, 'episodes_this_iter': 43, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], 'episode_lengths': [89, 87, 77, 94, 107, 85, 84, 96, 87, 92, 84, 88, 112, 94, 108, 97, 107, 97, 82, 84, 73, 99, 89, 102, 100, 123, 102, 107, 101, 92, 88, 84, 102, 112, 79, 83, 104, 113, 96, 93, 79, 86, 82, 95, 77, 91, 71, 139, 89, 67, 74, 83, 100, 92, 94, 101, 86, 107, 82, 85, 100, 95, 130, 80, 76, 78, 80, 99, 76, 74, 93, 110, 83, 121, 93, 98, 102, 96, 101, 85, 132, 70, 117, 93, 77, 99, 128, 100, 91, 85, 94, 70, 86, 66, 81, 95, 99, 117, 79, 94]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.4404442296553175, 'mean_inference_ms': 1.2645802029971827, 'mean_action_processing_ms': 0.1045043056800169, 'mean_env_wait_ms': 0.0883236720705588, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.004522562026977539, 'StateBufferConnector_ms': 0.0033309459686279297, 'ViewRequirementAgentConnector_ms': 0.2528705596923828}}","{'training_iteration_time_ms': 15927.75, 'sample_time_ms': 7490.866, 'synch_weights_time_ms': 2.538}"


2023-08-18 22:44:57,940	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-08-18 22:44:58,585	INFO tune.py:1148 -- Total run time: 1947.66 seconds (1924.73 seconds for the tuning loop).
2023-08-18 22:44:58,587	WARNING tune.py:1163 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)


In [6]:
from ray.rllib.algorithms.algorithm import Algorithm

algo2 = Algorithm.from_checkpoint("/workspaces/codespaces-jupyter/notebooks/Urrnow/PPO/PPO_my_env_60598_00000_0_2023-08-18_22-12-53/checkpoint_000100")

2023-08-18 22:45:19,281	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-18 22:45:19,282	WARNING algorithm_config.py:656 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2023-08-18 22:45:19,284	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.e

In [ ]:
try:
    import gymnasium as gym
    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = "my_env"
env = GoLeftEnv()
#algo = PPOConfig().environment(env_name).rollouts(num_rollout_workers=1).build()
iii = 0
while iii<100:
  iii = iii+1
  episode_reward = 0
  terminated = truncated = False

  if gymnasium:
      obs, info = env.reset()
  else:
      obs = env.reset()

  while not terminated and not truncated:
      action = algo2.compute_single_action(obs)
      if gymnasium:
          obs, reward, terminated, truncated, info = env.step(action)
      else:
          obs, reward, terminated, info = env.step(action)
      episode_reward += reward
  print(episode_reward)